# # Modelagem 1-dim | 128 mfccs

In [ ]:
import keras as K
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from random import choice

from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report

%matplotlib inline

In [ ]:
DIR_DATAFRAMES = '/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/DATAFRAMES'
DIR_MFCC = '/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/MFCC'

In [ ]:
!ls -lah $DIR_DATAFRAMES

In [ ]:
!ls -lahR $DIR_MFCC

In [ ]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
DIR_DATAFRAMES = '/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/DATAFRAMES'
DIR_MFCC = '/media/hbrandao/STORAGE_HDD_1TB/PPGI/BRAVO-DATAFRAMES/MFCC'

In [ ]:
!ls -lahR $DIR_MFCC

In [ ]:
PHYS_DEVICE_NAME = tf.config.list_physical_devices('GPU')[0].name[-5:]
if PHYS_DEVICE_NAME:
    print(PHYS_DEVICE_NAME)
else:
    raise Exception('GPU não encontrada')

## Dados
___

In [ ]:
X128 = np.load(f'{DIR_MFCC}/mean/Xsinal_128mfcc_mean.npy')

x128train, x128test, *_ = train_test_split(X128)
x128train.shape, x128test.shape

## Autoencoder
___

In [ ]:
dim = X128.shape[-1]

In [ ]:
def autoencoder128():
    inp_enc = K.Input(batch_shape=(1,dim))
    enc = layers.Dense(units=int(dim/2), activation='relu')(inp_enc)
    encoder = K.Model(inp_enc, enc)

    inp_dec = K.Input(batch_shape=(1,int(dim/2)))
    dec = layers.Dense(units=dim, activation='linear')(inp_dec)
    decoder = K.Model(inp_dec, dec)

    inp_ae = K.Input(batch_shape=(1,dim))
    enc_ae = encoder(inp_ae)
    dec_ae = decoder(enc_ae)

    ae = K.Model(inp_ae, dec_ae, name='autoencoder')
    ae.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return ae, encoder, decoder

### x128 train

In [ ]:
epochs = 1000

es = EarlyStopping(monitor='loss', mode='min', min_delta=0.01, patience=20, verbose=1)

# mc = ModelCheckpoint('autoencoder_mean_128mfcc.hdf5',
#                      monitor='loss', verbose=1, save_best_only=True, mode='min')

with tf.device(PHYS_DEVICE_NAME):
    K.backend.clear_session()
    modelo128, encoder128, decoder128 = autoencoder128()
    history = modelo128.fit(x=x128train,
                           y=x128train,
                           epochs=epochs,
                           batch_size=10,
                           shuffle=True,
                           validation_data=(x128test, x128test),
                           callbacks=[es]#, mc]
                          )

In [ ]:
plt.figure(figsize=(14, 12))

row, col = 2, 2

for i,r in enumerate((0, 50, 100, 150)):
    plt.subplot(row,col,i+1)
    plt.plot(history.history['loss'][r:])
    plt.plot(history.history['val_loss'][r:])
    plt.title(f'Loss[{r}:]')
    plt.legend(['train loss', 'val loss'], loc='upper right')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,8))

row, col, idx = 2, 2, 1

for i in range(row*col):
    reg = choice(x128train).reshape(1,dim)
    pred = modelo128.predict(reg)
    diff = reg - pred

    plt.subplot(row, col, idx)
    plt.plot(reg.flatten(), label='amostra')
    plt.plot(pred.flatten(), label='predict', linestyle='--')
    plt.plot(diff.flatten(), label='amostra - pred')
    plt.legend()
    plt.grid()
    idx += 1

plt.tight_layout()
plt.show()

In [ ]:
pred_x128 = modelo128.predict(x128test)
loss_x128 = K.losses.mse(pred_x128, x128test)

np.mean(loss_x128)

In [ ]:
#
#
modelo128 = K.models.load_model('autoencoder_mean_128mfcc.hdf5')
#
#

## Classificador

In [ ]:
X128.shape

In [ ]:
Xfusao = pd.read_csv(f'{DIR_DATAFRAMES}/Xfusao.csv', delimiter=';')
Xfusao.shape

In [ ]:
DE_PARA_EMOCAO = {
    'alegria': np.array([1,0,0,0]),
    'medo': np.array([0,1,0,0]),
    'raiva': np.array([0,0,1,0]),
    'surpresa': np.array([0,0,0,1]),
}

DE_PARA_INTENSIDADE = {
    'fraca': np.array([1,0,0,0]),
    'moderada': np.array([0,1,0,0]),
    'forte': np.array([0,0,1,0]),
    'pico': np.array([0,0,0,1]),
}

def encoding_emocao(emocao: str):
    return DE_PARA_EMOCAO.get(emocao)

def encoding_intensidade(intensidade: str):
    return DE_PARA_INTENSIDADE.get(intensidade)

In [ ]:
Xfusao['emo'] = Xfusao['emocao'].apply(encoding_emocao)
Xfusao['int'] = Xfusao['intensidade'].apply(encoding_intensidade)

In [ ]:
Xfusao.drop(labels=['path', 'pessoa', 'genero', 'duracao', 'emocao', 'intensidade'],
            axis=1, inplace=True)

In [ ]:
Xfusao = pd.concat(objs=[Xfusao, pd.DataFrame(X128)], axis=1)
Xfusao.tail()

In [ ]:
Xfusao_verbo = Xfusao[Xfusao['dataset'] == 'verbo']
Xfusao_verbo.shape

In [ ]:
Xfusao_vivae = Xfusao[Xfusao['dataset'] == 'vivae']
Xfusao_vivae.shape

In [ ]:
def realizar_encoding(x, dim=dim):
    return encoder128(x.reshape(1,dim).astype('float32'))

In [ ]:
x128_vivae_enc = np.array([realizar_encoding(x) for x in Xfusao_vivae.values[:,4:]])
x128_vivae_enc.shape

In [ ]:
y128_vivae = np.array([x.reshape(1,4) for x in Xfusao_vivae['int'].values])
y128_vivae.shape

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x128_vivae_enc, y128_vivae)

xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

### Modelo

In [ ]:
def modelo_clf_intensidade():
    K.backend.clear_session()
    modelo = K.Sequential()
    modelo.add(layers.Dense(units=32, activation='relu', input_shape=(1,64)))
    modelo.add(layers.Dense(units=16, activation='relu'))
    modelo.add(layers.Dense(units=8, activation='relu'))
    modelo.add(layers.Dense(units=4, activation='softmax'))

    modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    print(modelo.summary())
    return modelo

modelo_clf_intensidade()

In [ ]:
epochs = 1000

es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.001, patience=20, verbose=1)

# mc = ModelCheckpoint('classificador_intensidade_128mfcc_encoded.hdf5',
#                      monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


with tf.device(PHYS_DEVICE_NAME):
    K.backend.clear_session()
    clf_intensidade = modelo_clf_intensidade()
    history = clf_intensidade.fit(x=xtrain,
                                  y=ytrain,
                                  epochs=epochs,
                                  batch_size=2,
                                  shuffle=True,
                                  validation_data=(xtest, ytest),
                                  callbacks=[es]#, mc]
                                 )

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train loss', 'val loss'], loc='upper right')
plt.title('Classificador de Intensidade')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
_ = classification_report(
    y_true=[np.argmax(x) for x in ytest],
    y_pred=[np.argmax(x) for x in clf_intensidade.predict(xtest)],
    target_names=DE_PARA_INTENSIDADE.keys()
)

print(_)

In [ ]:
#
#
clf_intensidade = K.models.load_model('classificador_intensidade_128mfcc_encoded.hdf5')
#
#

In [ ]:
_ = classification_report(
    y_true=[np.argmax(x) for x in ytest],
    y_pred=[np.argmax(x) for x in clf_intensidade.predict(xtest)],
    target_names=DE_PARA_INTENSIDADE.keys(),
    zero_division=0
)

print(_)

#### VERBO

In [ ]:
xtest_verbo = np.array([realizar_encoding(x) for x in Xfusao_verbo.values[:,4:]])
pred_verbo = [np.argmax(x) for x in clf_intensidade.predict(xtest_verbo)]

In [ ]:
np.array(pred_verbo)
xtest_verbo = np.array([realizar_encoding(x) for x in Xfusao_verbo.values[:,4:]])
pred_verbo = [np.argmax(x) for x in clf_intensidade.predict(xtest_verbo)]
In [42]:


## Decomposição

In [ ]:
LABEL_INTENSIDADE = {
    0: 'fraca',
    1: 'moderada',
    2: 'forte',
    3: 'pico'
}

In [ ]:
pred_verbo_label = [LABEL_INTENSIDADE.get(_) for _ in pred_verbo]

In [ ]:
pred_vivae_label = [
        LABEL_INTENSIDADE.get(np.argmax(x)) for x in clf_intensidade.predict(x128_vivae_enc)
]

In [ ]:
x128_vivae_enc.shape, xtest_verbo.shape

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

### PCA

@VIVAE MFCCs

In [ ]:
pca = PCA(n_components=2)
Xpca = pca.fit_transform(Xfusao_vivae.values[:,4:])
Xpca.shape

In [ ]:
df_pca = pd.DataFrame(data=Xpca, columns=('c1', 'c2'))
df_pca['label'] = [LABEL_INTENSIDADE.get(np.argmax(x)) for x in y128_vivae]
df_pca.head()

In [ ]:
plt.figure(figsize=(16, 10))

sns.scatterplot(data=df_pca, x='c1', y='c2', hue='label')
plt.suptitle('PCA aplicado na média dos 64 MFCCs de VIVAE', fontsize=24)
plt.title(f'{df_pca.shape[0]} registros', fontsize=18)
plt.xlabel('componente 1')
plt.ylabel('componente 2')

plt.show()

@VIVAE AE

In [ ]:
pca = PCA(n_components=2)
Xpca = pca.fit_transform([*x128_vivae_enc.reshape(698,64)])
Xpca.shape

In [ ]:
df_pca = pd.DataFrame(data=Xpca, columns=('c1', 'c2'))
df_pca['label'] = [LABEL_INTENSIDADE.get(np.argmax(x)) for x in y128_vivae]
df_pca.head()

In [ ]:
plt.figure(figsize=(16, 10))

sns.scatterplot(data=df_pca, x='c1', y='c2', hue='label')
plt.suptitle('PCA aplicado ao resultado do AE em VIVAE', fontsize=24)
plt.title(f'{df_pca.shape[0]} registros', fontsize=18)
plt.xlabel('componente 1 - dado encoded')
plt.ylabel('componente 2 - dado encoded')

plt.show()

@AE VERBO e VIVAE

In [ ]:
pca = PCA(n_components=2)
Xpca = pca.fit_transform([*x128_vivae_enc.reshape(698,64), *xtest_verbo.reshape(666,64)])
Xpca.shape

In [ ]:
df_pca = pd.DataFrame(data=Xpca, columns=('c1', 'c2'))
df_pca['label'] = [*pred_vivae_label, *[_ + ' (verbo)' for _ in pred_verbo_label]]
df_pca.head()

In [ ]:
plt.figure(figsize=(16, 10))

sns.scatterplot(data=df_pca, x='c1', y='c2', hue='label')
plt.suptitle('PCA aplicado ao resultado do AE em VERBO e VIVAE', fontsize=24)
plt.title(f'{df_pca.shape[0]} registros', fontsize=18)
plt.xlabel('componente 1 - dado encoded')
plt.ylabel('componente 2 - dado encoded')

plt.show()

@AE VERBO e VIVAE - 2 Classes

In [ ]:
DE_PARA_LABEL_2_CLASSES = {
    'fraca': 'baixa',
    'moderada': 'baixa',
    'forte': 'alta',
    'pico': 'alta',

    'fraca (verbo)': 'baixa (verbo)',
    'moderada (verbo)': 'baixa (verbo)',
    'forte (verbo)': 'alta (verbo)',
    'pico (verbo)': 'alta (verbo)',
}


df_pca['label 2'] = df_pca['label'].apply(lambda x: DE_PARA_LABEL_2_CLASSES.get(x))

In [ ]:
plt.figure(figsize=(16, 10))

sns.scatterplot(data=df_pca, x='c1', y='c2', hue='label 2')
plt.suptitle('PCA aplicado ao resultado do AE em VERBO e VIVAE - 2 classes', fontsize=24)
plt.title(f'{df_pca.shape[0]} registros', fontsize=18)
plt.xlabel('componente 1 - dado encoded')
plt.ylabel('componente 2 - dado encoded')

plt.show()